In [ ]:
### Raw CBI raster stack

#!/usr/bin/env python3
import rasterio
import numpy as np
from rasterio.windows import Window

cbi_raster = "data/derived/not_resampled/wildfire_cbi.tif"
forest_mask = "data/derived/resampled/forest_mask_30m_resampled.tif"
out_raster = "tmp/cbi_clean/cbi_2000_2020_burned.tif"

with rasterio.open(cbi_raster) as cbi, rasterio.open(forest_mask) as fm:

    meta = cbi.meta.copy()
    meta.update(
        count=1,
        dtype=rasterio.uint8,
        compress="DEFLATE",
        nodata=0,
        BIGTIFF="YES"
    )

    with rasterio.open(out_raster, "w", **meta) as dst:

        total_burned_pixels = 0
        pixel_area_km2 = (30 * 30) / 1e6

        # Process in blocks
        for ji, window in cbi.block_windows(1):
            # Initialize collapsed block
            collapsed = np.zeros((window.height, window.width), dtype=np.uint8)

            # Read forest mask block
            forest_block = fm.read(1, window=window)
            forest_block = (forest_block == 1).astype(np.uint8)

            # Loop over all 21 bands
            for b in range(1, cbi.count + 1):
                band = cbi.read(b, window=window)
                burned = (band > 0.1).astype(np.uint8)
                burned *= forest_block
                collapsed = np.where(burned == 1, 1, collapsed)

            dst.write(collapsed, 1, window=window)

            # accumulate stats
            total_burned_pixels += np.count_nonzero(collapsed == 1)

        total_area_km2 = total_burned_pixels * pixel_area_km2

print("========== 2000–2020 CBI FOOTPRINT ==========")
print("Burned pixels:", total_burned_pixels)
print("Burned area (km²):", round(total_area_km2, 2))
print("Output raster:", out_raster)
print("============================================")

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

raster_path = "tmp/cbi_clean/cbi_2000_2020_burned.tif"

with rasterio.open(raster_path) as src:
    data = src.read(1)

# Mask background
masked = np.ma.masked_where(data == 0, data)

plt.figure(figsize=(10, 12))
plt.imshow(masked, interpolation="nearest")
plt.title("Forest Burn Footprint (CBI > 0.1, 2000–2020)")
plt.axis("off")
plt.show()


In [ ]:
### Subset, resampled, masked annual rasters

import rasterio
import numpy as np
import os

# Paths
base_path = "data/derived/resampled"
file_template = "wildfire_cbi_30m_resampled_masked_{year}.tif"
output_path = "tmp/cbi_clean/cbi_2000_2020_masked_collapsed.tif"

years = range(2000, 2021)

# Initialize
collapsed = None
meta = None

for i, year in enumerate(years, 1):
    fpath = os.path.join(base_path, file_template.format(year=year))
    print(f"Reading {fpath} ({i}/{len(years)})")
    
    with rasterio.open(fpath) as src:
        data = src.read(1)
        if meta is None:
            meta = src.meta.copy()
            meta.update(dtype=rasterio.uint8, count=1, compress='DEFLATE', nodata=0)
        
        # Binarize: 1 if CBI > 0.1, 0 otherwise
        binary = (data > 0.1).astype(np.uint8)
        
        if collapsed is None:
            collapsed = binary
        else:
            collapsed = np.where((collapsed == 1) | (binary == 1), 1, 0)

# Save collapsed footprint
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with rasterio.open(output_path, 'w', **meta) as dst:
    dst.write(collapsed, 1)

# Calculate burned area
burned_px = np.count_nonzero(collapsed == 1)
pixel_area_km2 = (30*30)/1e6  # 30 m × 30 m pixels
burned_area_km2 = burned_px * pixel_area_km2

print("\n========== 2000–2020 CBI FOOTPRINT (MASKED) ==========")
print("Burned pixels:", burned_px)
print("Burned area (km²):", burned_area_km2)
print("Output raster:", output_path)
print("=============================================\n")


In [ ]:
### Annual stacks

import rasterio
import numpy as np
from rasterio.enums import Resampling

# Paths and output
years = range(2000, 2021)
stack_template = "data/derived/annual_stacks/annual_stack_{}.tif"
output_raster = "tmp/cbi_clean/cbi_2000_2020_wf_from_annuals.tif"

# Temporary variables
collapsed = None

for i, yr in enumerate(years):
    f = stack_template.format(yr)
    print(f"Processing {f} ...")
    
    with rasterio.open(f) as src:
        # Read first band (wildfire / CBI)
        wf_band = src.read(1)
        
        # Binarize (CBI > 0.1)
        wf_bin = (wf_band > 0.1).astype(np.uint8)
        
        if collapsed is None:
            # Initialize cumulative array
            collapsed = wf_bin
            meta = src.meta.copy()
            # --- FIX: set nodata compatible with uint8 ---
            meta.update(
                dtype=rasterio.uint8,
                count=1,
                compress='DEFLATE',
                nodata=0,  # previously was -9999 -> causes ValueError
                tiled=True,
                BIGTIFF='YES'
            )
        else:
            # Collapse by logical OR
            collapsed = ((collapsed > 0) | (wf_bin > 0)).astype(np.uint8)

# Write cumulative raster
with rasterio.open(output_raster, 'w', **meta) as dst:
    dst.write(collapsed, 1)

# Calculate area
burned_px = np.count_nonzero(collapsed)
pixel_area_km2 = (30*30)/1e6  # 30 m resolution
area_km2 = burned_px * pixel_area_km2

print("\n========== 2000–2020 CBI FOOTPRINT (ANNUAL STACKS) ==========")
print("Burned pixels:", burned_px)
print("Burned area (km²):", area_km2)
print("Output raster:", output_raster)
print("=============================================")


In [ ]:
### Annual binary (any disturbance) rasters

import rasterio
import numpy as np
from rasterio.enums import Resampling

# Paths and output
years = range(2000, 2021)
stack_template = "data/derived/annual_stacks/binary/annual_stack_any_{}.tif"
output_raster = "tmp/cbi_clean/cbi_2000_2020_wf_from_binary_annuals.tif"

# Temporary variables
collapsed = None

for i, yr in enumerate(years):
    f = stack_template.format(yr)
    print(f"Processing {f} ...")
    
    with rasterio.open(f) as src:
        # Read first band (wildfire / already binarized)
        wf_bin = src.read(1).astype(np.uint8)
        
        if collapsed is None:
            # Initialize cumulative array
            collapsed = wf_bin
            meta = src.meta.copy()
            # --- FIX: set nodata compatible with uint8 ---
            meta.update(
                dtype=rasterio.uint8,
                count=1,
                compress='DEFLATE',
                nodata=0,
                tiled=True,
                BIGTIFF='YES'
            )
        else:
            # Collapse by logical OR
            collapsed = ((collapsed > 0) | (wf_bin > 0)).astype(np.uint8)

# Write cumulative raster
with rasterio.open(output_raster, 'w', **meta) as dst:
    dst.write(collapsed, 1)

# Calculate area
burned_px = np.count_nonzero(collapsed)
pixel_area_km2 = (30*30)/1e6  # 30 m resolution
area_km2 = burned_px * pixel_area_km2

print("\n========== 2000–2020 WILDFIRE FOOTPRINT (BINARY ANNUAL STACKS) ==========")
print("Burned pixels:", burned_px)
print("Burned area (km²):", area_km2)
print("Output raster:", output_raster)
print("==============================================================")

In [ ]:

with rasterio.open("data/derived/annual_stacks/binary/annual_stack_any_2005.tif") as src:
    b1 = src.read(1)
    print(np.unique(b1, return_counts=True))


with rasterio.open("data/derived/annual_stacks/annual_stack_2005.tif") as src:
    wf = (src.read(1) > 0.1).astype(np.uint8)
    print(np.unique(wf, return_counts=True))